In [1]:
import sys
#sys.path.insert(0,'/home/rick/transformers/src')
import os
from pathlib import Path

trulens_path = (Path(os.getcwd()).parent.parent / "trulens")

sys.path.insert(0, str(trulens_path))
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer
import torch


mname="facebook/blenderbot-3B"
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
device="cuda:0"
model.to(device)
tok=tokenizer

In [5]:
import copy
# Taken as a subset of the methods that do model input prep from transformers.generation.utils
def prepare_inputs(inputs):
    generation_config = copy.deepcopy(model.generation_config)

    model_kwargs = generation_config.update(**inputs)
    inputs_tensor, model_input_name, model_kwargs = model._prepare_model_inputs(
        None, generation_config.bos_token_id, model_kwargs
    )
    batch_size = inputs_tensor.shape[0]
    model_kwargs["output_attentions"] = generation_config.output_attentions
    model_kwargs["output_hidden_states"] = generation_config.output_hidden_states
    model_kwargs["use_cache"] = generation_config.use_cache
    model_kwargs["attention_mask"] = model._prepare_attention_mask_for_generation(
        inputs_tensor, generation_config.pad_token_id, generation_config.eos_token_id
    )

    #model_kwargs = model._prepare_encoder_decoder_kwargs_for_generation(
    #                inputs_tensor, model_kwargs, model_input_name
    #            )

    input_ids = model._prepare_decoder_input_ids_for_generation(
        batch_size,
        decoder_start_token_id=generation_config.decoder_start_token_id,
        bos_token_id=generation_config.bos_token_id,
        model_kwargs=model_kwargs,
        device=inputs_tensor.device,
    )


    #input_ids, model_kwargs = model._expand_inputs_for_generation(
    #    input_ids=input_ids,
    #    expand_size=generation_config.num_beams,
    #    is_encoder_decoder=model.config.is_encoder_decoder,
    #    **model_kwargs,
    #)

    model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
    model_inputs['input_ids']=inputs['input_ids'] # NEW
    return model_inputs

In [6]:
from typing import Iterable, Union, Optional
from truera.nlp.general.aiq.nlp_coloring import attributions_to_rgb
from truera.nlp.general.aiq.nlp_coloring import generate_rgb_str
from truera.nlp.general.aiq.nlp_coloring import MAX_INTENSITY
from truera.nlp.general.aiq.nlp_coloring import rgb_str
from IPython.display import HTML
# copied from truera
def _influence_examples(
        tokens_list: Iterable[Iterable[str]],
        attributions_list: Iterable[Iterable[float]],
        *,
        qoi_class: Union[int, str] = 0,
        underline_list: Optional[Union[Iterable[Iterable[int]],
                                       Iterable[int]]] = None,
        prepends: Union[Iterable[str], str] = ''
    ) -> HTML:
        """
        plot the tokens & their attributions for list of token and attributions one by one
        underline_list specify the token index to underline (used for plotting influence of a sentence
        containing a specific token)
        """
        if len(attributions_list) == 0:
            return
        norm_factor = np.max(
            [
                np.max(np.abs(attributions))
                for attributions in attributions_list
            ]
        )
        # Display legend
        neg_infl_color = rgb_str(256, 256 - MAX_INTENSITY, 256 - MAX_INTENSITY)
        neutral_color = rgb_str(256, 256, 256)
        pos_infl_color = rgb_str(256 - MAX_INTENSITY, 256, 256 - MAX_INTENSITY)
        if isinstance(qoi_class, int):
            qoi_class = f"Class: {qoi_class}"
        qoi_class = qoi_class.replace('_', ' ')
        qoi_class = qoi_class.title()
        html_str = [
            f'''
            <div style="margin:auto; width:50%; height:20px; display:flex; align-items:center;justify-content: space-between; background-image:linear-gradient(to right, {neg_infl_color}, {neutral_color}, {pos_infl_color});">
                <strong style=margin-left:4px>Negative Influence</strong>
                <strong style=text-align:center>{qoi_class}</strong>
                <strong style=margin-right:4px>Postive Influence</strong>
            </div>
            '''
        ]

        # Plot examples
        if isinstance(prepends, str):
            prepends = [prepends for _ in range(len(tokens_list))]
        for si, (attributions, tokens, prepend) in enumerate(
            zip(attributions_list, tokens_list, prepends)
        ):
            underline_idxs = [
                underline_list[si]
            ] if underline_list is not None else None
            if isinstance(underline_idxs, int):
                underline_idxs = [underline_idxs]
            line_html = generate_rgb_str(
                tokens,
                attributions,
                underline_idxs,
                norm_factor=norm_factor,
                max_intensity=MAX_INTENSITY
            )
            html = f'<p style=padding-bottom:2px><h4 style=margin:0;>{prepend}</h4> {line_html}'
            html_str.append(html)

        return HTML("\n".join(html_str))

In [7]:
from trulens.nn.models import get_model_wrapper
from trulens.nn.distributions import PointDoi
from trulens.nn.quantities import LambdaQoI
from trulens.nn.attribution import IntegratedGradients, Saliency
from trulens.nn.attribution import Cut, OutputCut,InputCut
from trulens.utils.typing import ModelInputs
import numpy as np
def explain(inputs, model_inputs, reply_ids):
    chosen_token = int(reply_ids[0][1])
    #chosen_token = 298
    wrapper = get_model_wrapper(model)
    qoi = LambdaQoI(lambda out: out[-1][-1][chosen_token])
    infl = Saliency(
            model=wrapper,
            cuts=(Cut('layers_0_self_attn_q_proj', anchor='in'), OutputCut(accessor=lambda o: o['logits'])),
            #qoi_cut=OutputCut(accessor=lambda o: o['logits']),
            qoi=qoi)
    infl = infl.attributions(**model_inputs)
    influence_sums = np.sum(infl[0][0].cpu().detach().numpy()[0], axis = 1)
    input_tokens = tokenizer.batch_decode(inputs['input_ids'][0])
    display(_influence_examples([input_tokens], [influence_sums], prepends=f"next token:{tokenizer.decode(chosen_token)}"))

In [8]:
import copy
def explain_utterance(utterance, reply_ids):
    #for i in range(len(reply_ids)-1):
    #    next_token = i+1
        
    inputs = tokenizer([utterance], return_tensors="pt").to(device)
    
    model_inputs = prepare_inputs(inputs)
    explain(inputs, model_inputs, reply_ids)


In [12]:
from transformers..configuration_utils import GenerationConfig
GenerationConfig.from_model_config(self.config)

NameError: name 'GenerationConfig' is not defined

In [10]:
UTTERANCE = "<s> What Tools are used in baseball?. "
inputs = tokenizer([UTTERANCE], return_tensors="pt").to(device)
reply_ids = model.generate(**inputs)
print([f"{tok_id}: {tokenizer.decode(tok_id)}" for tok_id in reply_ids[0]])
print(tokenizer.batch_decode(reply_ids))
#print(reply_ids)
explain_utterance(UTTERANCE, reply_ids)

['1: <s>', '349:  A', '1703:  bat', '19: ,', '265:  a', '3561:  glo', '309: ve', '19: ,', '298:  and', '265:  a', '1703:  bat', '581: ting', '3561:  glo', '309: ve', '21: .', '1: <s>', '228:  ', '1: <s>', '1: <s>', '1: <s>', '2: </s>']
['<s> A bat, a glove, and a batting glove.<s> <s><s><s></s>']


AttributeError: 'BlenderbotForConditionalGeneration' object has no attribute 'generation_config'

In [ ]:
raise Exception("below is experimental beam explanations")

In [ ]:
UTTERANCE = "<s> What Tools are used in baseball?. </s> <s> A bat, a glove, and a "
inputs = tokenizer([UTTERANCE], return_tensors="pt").to(device)
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))
print(reply_ids)
explain_utterance(UTTERANCE, reply_ids)

In [ ]:
NEXT_UTTERANCE = (
...     "<s> What Tools are used in baseball?.</s> "
...     "<s> A bat, a glove, a helmet, and a ball.</s> "
...     "<s> what about hockey? "
... )
inputs = tokenizer([NEXT_UTTERANCE], return_tensors="pt").to(device)
reply_ids = model.generate(**inputs)
print([f"{tok_id}: {tokenizer.decode(tok_id)}" for tok_id in reply_ids[0]])
print(tokenizer.batch_decode(reply_ids))
explain_utterance(NEXT_UTTERANCE, reply_ids)

In [ ]:
NEXT_UTTERANCE = (
...     "<s> What Tools are used in baseball?.</s> "
...     "<s> A bat, a glove, a helmet, and a ball.</s> "
...     "<s> what about hockey? <s> Hockey is a team sport played on ice, usually with a "
... )
inputs = tokenizer([NEXT_UTTERANCE], return_tensors="pt").to(device)
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))
print(reply_ids)
explain_utterance(NEXT_UTTERANCE, reply_ids)